# Comparing Annual Changes in Sea Surface Temperature and Polar Sea Ice Concentrations 

## EDS 220, Fall 2021



## Authors

- **Peter Menzies**, UC Santa Barbara (petermenzies@bren.ucsb.edu) <br>
https://petermenzies.github.io/


- **Julie Cohen**, UC Santa Barbara (jscohen@bren.ucsb.edu) <br>
https://julietcohen.github.io/


- **Ryan Munnikhuis**, UC Santa Barbara (Rmunnikhuis@ucsb.edu) <br>
https://ryanmunnikhuis.github.io/


## Table of Contents

[1. Purpose](#purpose)

[2. Dataset Description](#overview)

[3. Data I/O](#io)

[4. Metadata Display and Basic Visualization](#display)

[5. Use Case Examples](#usecases)

[6. Create Binder Environment](#binder)

[7. References](#references)

<a id='purpose'></a> 
### Notebook Purpose

This notebook is intended to introduce you into working with, subsetting, and visualizing oceanic and ice geodatasets. For this exercise, we'll be subsetting and visualsing sea surface temperatures (SST) from the NOAA Optimum Interpolation 1/4 Degree Daily Sea Surface Temperature (OISST) Analysis, Version 2) and sea ice concentrations (SIC) data from the Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1) data set. 

At the end of the lab, we will ideally have: 

- A gif showing changes of sea surface temperatures over a 40-year timeframe; 


- A gif showing of changes of sea ice concentrations over a 40-year timeframe; and


- A graph showing the trend in sea surface temperatures and sea ice concentrations over time.

<a id='overview'></a> 
### Datasets

In this lab, we'll be using data from two sources. 
- SST: NOAA's 1/4° Daily Optimum Interpolation Sea Surface Temperature (OISST) version 2, or OISSTv2
- SIC: Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1)

Data Google Drive: https://drive.google.com/drive/folders/1bdVrH6EB0JAjreb0xAbcb6Nps5oyW2eW?usp=sharing

A description of each dataset is given below: 

### Sea Surface Temperature Data 

The NOAA 1/4° daily Optimum Interpolation Sea Surface Temperature (daily OISST) Climate Data Record (CDR) provides complete ocean temperature fields constructed by combining bias-adjusted observations from different platforms (satellites, ships, buoys) on a regular global grid, with gaps filled in by interpolation. The main input source is satellite data from the Advanced Very High-Resolution Radiometer (AVHRR), which provides high temporal-spatial coverage from late 1981 to–present. 
This dataset is stored as NetCDF files (.nc files) that include latitude and longitude coordinates, temperatures, and dates.

Data source and metadata: NOAA National Centers for Environemntal Information https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00844/html

NOAA Sea Surface Temperature Optimum Interpolation methods: https://www.ncei.noaa.gov/products/climate-data-records/sea-surface-temperature-optimum-interpolation


### Sea Ice Concentration 

The Monthly Mean Hadley Centre Sea Ice and SST dataset version 1 (HadISST1) is one half of the merged Hadley-OI sea surface temperature (SST) and sea ice concentration (SIC) dataset. The merged product provides monthly global mean sea surface temperature and sea ice concentration data from 1870 to the present. The merging procedure was designed to take full advantage of the higher-resolution SST information found in version 2 of the NOAA weekly optimum interpolation (OI) SST analysis, the other half of the merged dataset. The combined dataset blends historical SST and modern SST observations from ships, buoys, drifters, and sea ice observations, partly from historical ship- and air-borne and partly from satellite data.This dataset is stored as NetCDF files (.nc files) that include latitude and longitude coordinates, temperatures, and dates.

Data source and metadata: UCAR/NCAR - DASH Repository
https://dashrepo.ucar.edu/dataset/158_asphilli.html

Journal of Climate article explaining why the datsets were merged: https://journals.ametsoc.org/view/journals/clim/21/19/2008jcli2292.1.xml

## Dataset Input/Output

### Configure Environment

We will be using xarray, rasterio, pandas, matplotlib, rioxarray, imaegio, and glob Python packages.

In [ ]:
#Configure Python Environment
import xarray as xr
import rasterio as rio
import pandas as pd 
from rasterio.plot import show
import matplotlib.pyplot as plt
import rioxarray
import imageio
import glob

### Dataset Parameters 

We have preloaded subsetted data into binder to avoid binder size conflicts and to minimize time needed downloading data. We'll look at a 20 year timeframe for both sea surface temperatures and sea ice concentrations. Since we've already (briefly) gone over how to visualize SST in our last presentation, we'll just show you the code that and walk you through visualizing sea ice data.    

### Use Case Examples 

While static maps are useful for oberserving conditons at a single point in time (or an average of time span), it can sometimes be helpful to visual the change in geospatial data from year to year. This exercise is intended to provide you with multiple examples of how to animate a timeseries. In addition, this lab will also show you one method to plot two datasets on the same graph.  

## Part One: Sea Surface Temperature 

In [ ]:
# Example commands to open a single NetCDF file and view the metadata

# use xarray to load in the datafile
data_example = xr.open_dataset('Data/avhrr-only-v2.19810901.nc')
# use head() to view the metadata associated with this file
data_example.head()

In [ ]:
# List of years that data were collected to read in .nc files and to title maps later on
years = range(1981, 2022)

# Empty list to populate with tif file names
images = []

In [ ]:
# Loop to read in datasets and and export as rasterized tiffs
for i in years:
    
    dataset = 'Data/oisst-avhrr-v02r01.' + str(i) + '1017.nc'
    
    nc = xr.open_dataset(dataset)
    
    nc_projected = nc.rio.write_crs(4326, inplace=True)
    
    sst = nc_projected["sst"].rio.set_spatial_dims('lon', 'lat')
    
    sst = sst.rename({'lat':'latitude', 'lon':'longitude'})
    
    sst['time'] = pd.to_datetime(sst['time'], format='%Y%m%d')
    
    sst = sst.squeeze()
    
    filename = 'Data/noaa_sst_' + str(i) + '.tif'
    
    sst.rio.to_raster(filename)
    
    images.append(filename)

In [ ]:
# Function to render the tiffs we exported
def raster_show(raster):
    
    open_rast = rio.open(raster)
    
    plt.figure(figsize=(10,10), dpi=150)
    
    plt.imshow(open_rast.read(1),
               origin='lower',
               cmap='plasma',
               extent=[-180, 180, -90, 90])

In [ ]:
raster_show('Data/noaa_sst_1981.tif')

In [ ]:
raster_show('Data/noaa_sst_2021.tif')

In [ ]:
png_rasters = []

In [ ]:
 # Loop to export as png and create list of png file names
 for i in range(0, 41):
    raster_show(images[i])
    png = images[i] + '.png'
    plt.title(years[i])
    plt.savefig(png)
    png_rasters.append(png)

In [ ]:
# Created composite gif from pngs
png_imageio = []

for filename in png_rasters:
    png_imageio.append(imageio.imread(filename))
imageio.mimsave('Data/sst_81_21.gif', png_imageio, **{'duration':0.4})

## Part Two: Sea Ice Concentrations 

In [ ]:
ds_ice = xr.open_dataset("Data/MODEL.ICE.HAD187001-198110.OI198111-202109.nc")

In [ ]:
ice = ds_ice.rio.write_crs(4326, inplace=True)

In [ ]:
ice_time = pd.DataFrame(columns=['year', 'ice_conc'])

In [ ]:
ice_year = []

ice_conc = []

In [ ]:
ice_time['year'] = ice_year
ice_time['ice_conc'] = ice_conc

In [ ]:
ice_time

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(ice_time['year'], ice_time['ice_conc'])
plt.xlabel('Year')
plt.ylabel('Ice concentration')

In [ ]:
for i in range(0, 40):
    
    ice_i = float(ice['SEAICE'][1341 + (12 * i)].mean())

    year_i = 1981 + i

    ice_year.append(year_i)

    ice_conc.append(ice_i)

## Part Three: Comparing Datasets 

<a id='usecases'></a> 
### Use Case Examples

This is the "meat" of the notebook, and what will take the majority of the time to present in class. This section should provide:
1) A plain-text summary (1-2 paragraphs) of the use case example you have chosen: include the target users and audience, and potential applicability. For example, the Week 7 SST exercise might discuss how the state of the ENSO system can be important for seasonal weather forecasts/coral bleaching outlooks, then mention the typical diagnostics associated with ENSO (i.e. identification of El Nino/La Nina events).

2) Markdown and code blocks demonstrating how one walks through the desired use case example. This should be similar to the labs we've done in class: you might want to demonstrate how to isolate a particularly interesting time period, then create an image showing a feature you're interested in, for example.

3) A discussion of the results and how they might be extended on further analysis. For example, we are doing El Nino/La Nina composites in class; a natural extension might be to look at individual events to see what their particular impacts were. Or if there are data quality issues which impact the results, you could discuss how these might be mitigated with additional information/analysis.

Just keep in mind, you'll have roughly 20 minutes for your full presentation, and that goes surprisingly quickly! Probably 2-3 diagnostics is the most you'll be able to get through (you could try practicing with your group members to get a sense of timing).


<a id='binder'></a> 
### Create Binder Environment

The last step is to create a Binder environment for your project, so that we don't have to spend time configuring everyone's environment each time we switch between group presentations. Instructions are below:

 - Assemble all of the data needed in your Github repo: Jupyter notebooks, a README file, and any datasets needed (these should be small, if included within the repo). Larger datasets should be stored on a separate server, and access codes included within the Jupyter notebook as discussed above. 
 
 - Create an _environment_ file: this is a text file which contains information on the packages needed in order to execute your code. The filename should be "environment.yml": an example that you can use for the proper syntax is included in this template repo. To determine which packages to include, you'll probably want to start by displaying the packages loaded in your environment: you can use the command `conda list -n [environment_name]` to get a list.
 
 More information on environment files can be found here:
 https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#

 - Create Binder. Use http://mybinder.org to create a  URL for your notebook Binder (you will need to enter your GitHub repo URL). You can also add a Launch Binder button directly to your GitHub repo, by including the following in your README.md:

```
launch with myBinder
[![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/<path to your repo>)
```

<a id='references'></a> 
### References

List relevant references. Here are some additional resources on creating professional, shareable notebooks you may find useful:

1. Notebook sharing guidelines from reproducible-science-curriculum: https://reproducible-science-curriculum.github.io/publication-RR-Jupyter/
2. Guide for developing shareable notebooks by Kevin Coakley, SDSC: https://github.com/kevincoakley/sharing-jupyter-notebooks/raw/master/Jupyter-Notebooks-Sharing-Recommendations.pdf
3. Guide for sharing notebooks by Andrea Zonca, SDSC: https://zonca.dev/2020/09/how-to-share-jupyter-notebooks.html
4. Jupyter Notebook Best Practices: https://towardsdatascience.com/jupyter-notebook-best-practices-f430a6ba8c69
5. Introduction to Jupyter templates nbextension: https://towardsdatascience.com/stop-copy-pasting-notebooks-embrace-jupyter-templates-6bd7b6c00b94  
    5.1. Table of Contents (Toc2) readthedocs: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/nbextensions/toc2/README.html  
    5.2. Steps to install toc2: https://stackoverflow.com/questions/23435723/installing-ipython-notebook-table-of-contents
6. Rule A, Birmingham A, Zuniga C, Altintas I, Huang SC, et al. (2019) Ten simple rules for writing and sharing computational analyses in Jupyter Notebooks. PLOS Computational Biology 15(7): e1007007. https://doi.org/10.1371/journal.pcbi.1007007. Supplementary materials: example notebooks (https://github.com/jupyter-guide/ten-rules-jupyter) and tutorial (https://github.com/ISMB-ECCB-2019-Tutorial-AM4/reproducible-computational-workflows)
7. Languages supported by Jupyter kernels: https://github.com/jupyter/jupyter/wiki/Jupyter-kernels
8. EarthCube notebooks presented at EC Annual Meeting 2020: https://www.earthcube.org/notebooks
9. Manage your Python Virtual Environment with Conda: https://towardsdatascience.com/manage-your-python-virtual-environment-with-conda-a0d2934d5195
10. Venv - Creation of Virtual Environments: https://docs.python.org/3/library/venv.html